In [27]:
import pygal
import pandas as pd
import streamlit as st
from datetime import datetime

In [46]:
# Definindo a data mínima e máxima
data_minima = pd.to_datetime('2023-01-01')
data_maxima = pd.to_datetime('2024-05-23')

    
# Adicionar seletor de períodos na coluna à esquerda
start_date_ = st.sidebar.date_input("Data de início", data_minima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #"DD-MM-YYYY"

end_date_ = st.sidebar.date_input("Data de término", data_maxima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #+ pd.DateOffset(days=1) #"DD-MM-YYYY"

start_date = start_date_.strftime('%Y-%m-%d') #'%d-%m-%Y'
end_date = end_date_.strftime('%Y-%m-%d')

In [29]:
import csv

def encontrar_frase_em_csv_meta(nome_arquivo, frase_procurada):
    try:
        with open(nome_arquivo, 'r', newline='', encoding='utf-16') as arquivo_csv:
            leitor_csv = csv.reader(arquivo_csv)
            
            for numero_linha, linha in enumerate(leitor_csv, start=1):
                if frase_procurada in linha:
                    return numero_linha

        # Se a frase não for encontrada em nenhuma linha
        return -1

    except FileNotFoundError:
        print(f'O arquivo {nome_arquivo} não foi encontrado.')
        return -1

In [30]:
final_alcanceFB = encontrar_frase_em_csv_meta('tabelas/meta/alcance.csv', 'Alcance do Instagram')
dados_FB_alcance = pd.read_csv('tabelas/meta/alcance.csv',skiprows=2,encoding='utf-16',nrows=final_alcanceFB-5)
# dados_FB_alcance = pd.read_csv(r'C:\Users\Usuario\Documents\Repositórios\Dashboard-TN\tabelas\meta\alcance.csv',skiprows=3,nrows=10)

In [31]:
# Função para tentar múltiplos formatos
def try_parsing_date(text):
    for fmt in ("%Y-%m-%dT%H:%M:%S.%fZ", "%Y-%m-%d %H:%M:%S"):
        try:
            return pd.to_datetime(text, format=fmt)
        except ValueError:
            continue
    return pd.NaT

In [32]:
dados_FB_alcance['Data'] = dados_FB_alcance['Data'].apply(try_parsing_date)
dados_FB_alcance = dados_FB_alcance.dropna(subset=['Data'])

In [33]:
# Recebendo dados de 01/01/2022 até o momento
# alcance FB: 28/05/2022
# alcance IG: 28/05/2022
# visitas FB: 28/05/2022
# visitas IG: 28/05/2022
# seguidores FB: 01/01/2022
# seguidores IG: 31/08/2023
def filtrosDatasFACEBOOK(start_date, end_date):
    dados_FB_alcance_FILTRADAS = dados_FB_alcance.copy()
    
    dados_FB_alcance_FILTRADAS = dados_FB_alcance_FILTRADAS.loc[(dados_FB_alcance_FILTRADAS['Data']>start_date) & (dados_FB_alcance_FILTRADAS['Data']<end_date)]
    
    return dados_FB_alcance_FILTRADAS

In [43]:
#chamar a função com as datas

line_chart = pygal.Line()
line_chart.title = 'FB: Alcance'
line_chart.x_labels = map(str, range(1, start_date - end_date))
line_chart.add()
line_chart.render()


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [51]:
teste = end_date_ - start_date_
print(teste.days)

508
